# DEPRECATED

## The data we obtained from this api is not useful so we decided to continue to do it in other way

In [2]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import xml
import time

# Heroes

    We need the information of all heroes in the game in order to make a comparison, the data that we need are id, name, role and type.

**1. Make request to api asking for all heroes info**

In [98]:
res = req.get("https://api.opendota.com/api/heroes")

**2. Extract the useful data out of the response and store it on a DataFrame object**

In [102]:
heroes = pd.DataFrame(
    [{"id": hero["id"],
      "name": hero["localized_name"],
      "role": hero["roles"][0],
      "type": hero["attack_type"]} for hero in res.json()])

heroes.head()

,id,name,role,type
0,1,Anti-Mage,Carry,Melee
1,2,Axe,Initiator,Melee
2,3,Bane,Support,Ranged
3,4,Bloodseeker,Carry,Melee
4,5,Crystal Maiden,Support,Ranged


**3. Export the data to a csv file**

In [114]:
heroes.to_csv("../data/heroes.csv", index=False)

# Matches
    
    We need random samples of matches in order to compare drafts and winrates, one of the criterias I decided to use was MMR, higher MMR matches tend to be more consistend because the heroes are played next to its average potential which skews our data way less than if we use a bigger range of MMR, for this criteria I used an MMR equal or bigger to 3850 which is classified as Ancient rank or higher.

**1. Get random sample of matches based on a average mmr of 3850 or bigger**

In [106]:
res2 = req.get("https://api.opendota.com/api/publicMatches?mmr_descending=3850")

**2. check data received**

In [111]:
matches = pd.DataFrame(res2.json())
matches.head()

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,5496082216,4607663044,False,1593702504,1709,5817,2,7,22,80,6,191,"37,86,77,32,106","126,64,45,29,10"
1,5496083619,4607654399,True,1593702543,1262,5303,2,0,4,80,7,134,"71,48,49,84,104","73,44,2,100,67"
2,5496086210,4607674334,False,1593702667,2088,3669,1,7,22,80,4,138,"111,39,36,51,12","114,86,11,5,89"
3,5496088811,4607671955,True,1593702777,1829,4988,3,7,22,80,6,154,"83,39,70,111,65","67,126,99,128,21"
4,5496093104,4607683105,True,1593702950,2260,5415,7,7,3,80,10,236,"121,26,53,106,6","59,114,7,29,64"


**3. Create a function to retrieve matches data**
    - Must receive a number of requests argument
    - Cleans the format of the matches data
    - Returns a dataframe so we can store the data later to a .csv file

In [116]:
import time
import pandas as pd
import requests as req


def get_matches_sample(max_requests: int, folder: str, mmr=3850) -> pd.DataFrame:
    """
    Does a a number of requests to the api denoted by the max_requests argument.
    Returns a .csv file containing all the data found.
    NOTE: Does not group by unique findings.
    NOTE: Number of requests is limited by 50 000 per month
    """
    # data container
    matches_data = []
    count = 0
    while count < max_requests:
        # make request
        res = req.get(
            f"https://api.opendota.com/api/publicMatches?mmr_descending={mmr}")
        # check for errors in response
        if res.status_code == 200:
            # extend our data object with response obtained
            matches_data.extend(res.json())
        else:
            # break the loop and end the function
            print(f"There was an error with status_code: {res.status_code}")
            break
        count += 1
        # waits a second before continuing to limit the number of requests per minute to 60
        print(f"request n° {count}")
        time.sleep(1)
    # create DataFrame Object
    data = pd.DataFrame(matches_data)
    # correct format in heroes list
    data = data.replace(r"\,", "-", regex=True)
    # return our object!
    print("Saving file..")
    return data.to_csv(f"{folder}{int(time.time())}.csv", index=False)

**4. Execute the function**

In [117]:
get_matches_sample(2, "../rawdata/")

**5. Check that received data are correct**

In [4]:
pd.read_csv("../rawdata/1593961990.csv").head()

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,5496093104,4607683105,True,1593702950,2260,5415,7,7,3,80,10,236,121-26-53-106-6,59-114-7-29-64
1,5496093106,4607692069,False,1593702953,2576,3962,4,7,22,80,7,122,34-41-121-35-104,60-42-126-128-7
2,5496093306,4607678647,True,1593702953,1950,3625,1,7,22,80,4,133,7-104-6-79-23,100-126-3-108-1
3,5496095704,4607694805,True,1593703058,2394,7223,3,7,22,80,6,138,22-86-10-69-5,81-66-106-21-120
4,5496102206,4607686366,True,1593703331,1967,5626,5,7,22,80,8,156,41-110-36-13-120,9-14-32-10-53


**6. Export the data to a .csv file**

    The time function gives us the current Datetime in unix format, this way we can organize our data files by the date  we obtained them.

In [119]:
# data.to_csv(f"../rawdata/{int(time.time())}.csv", index=False)

**7. Loop our function to keep retrieving data until we are banned :D**

In [ ]:
count = 0

while count < 100:
    time.sleep(5)
    print(f"try n° {count}")
    get_matches_sample(10800, "rawdata/")
    count += 1